<a href="https://colab.research.google.com/github/subham1/sentence-transformers/blob/master/QuoraSentenceSimilarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import numpy as np
import pandas as pd

In [0]:
df =pd.read_csv('/train.csv')

In [3]:
cd ..


/


In [4]:
cd var 

/var


backups/  colab/  local/  log/   opt/  spool/
cache/    lib/    lock@   mail/  run@  tmp/


In [9]:
np.ones((2,2))

array([[1., 1.],
       [1., 1.]])

In [10]:
torch.ones(2,2)

tensor([[1., 1.],
        [1., 1.]])

In [13]:
torch.manual_seed(0)
torch.rand(2,2)

tensor([[0.4963, 0.7682],
        [0.0885, 0.1320]])

In [0]:
tmp = torch.ones(2,3)
if torch.cuda.is_available:
  tmp.cuda()